# Step 3: The Anecdote - When Evidence Accumulates

## 📅 8:15 AM - Five Hours Later

Remember the SIEM alert from Step_2? That was at 3:47 AM. The automated system detected the threat.

Now it's 8:15 AM. The CEO has arrived at work and found a message from the security team: "We detected a phishing email sent to you. Did you receive it?"

The CEO responds through your incident response portal:

> **User Report**  
> **From**: CEO  
> **Time**: 8:15 AM  
> **Message**: "Yes, I received an email this morning claiming to be from IT support. It asked me to verify my account by clicking a link. I didn't click it - it looked suspicious. The sender was something like 'attacker@evil.com' which clearly isn't our IT team."

This is an **anecdote** - a human report that confirms the automated detection.

## 🎯 Two Evidence Sources, One Incident

**Step_2** (Automated Detection):
- Source: SIEM system
- Evidence type: Sighting with `sighting-alert` extension
- Detection time: 3:47 AM
- Confidence: High (automated pattern matching)

**Step_3** (Human Verification):
- Source: CEO (victim)
- Evidence type: Sighting with `sighting-anecdote` extension
- Report time: 8:15 AM
- Confidence: Very High (first-hand witness)

Both point to **the same incident**. This is how incidents grow - evidence accumulates from multiple sources.

## 🗺️ The Journey: Three Acts

**Act 1: Setup** (Libraries, paths, utilities)

**Act 2: The User Report** (Anecdote creation)
- Pattern 3.9: Anecdote Provenance
- Create anecdote SCO (user's statement)
- Create sighting-anecdote (human verification)
- Create event (user-reported)

**Act 3: Impact Update** (Revised assessment)
- Pattern 3.10: Impact Supersession Chain
- Lower severity (user didn't click link)
- Supersede previous impact assessment

## 🎓 What You'll Learn

1. **Pattern 3.9 (Anecdote Provenance)**: How to record human testimony
2. **Pattern 3.10 (Impact Supersession)**: How impact assessments evolve
3. **Incident Extension**: How to add evidence to existing incidents
4. **Multi-Source Evidence**: How different sources confirm the same threat

## 📋 Prerequisites

✅ **Step_2_Create_Incident_with_an_Alert.ipynb** - You need the existing incident

## 🔜 What We'll Build

- 1× anecdote SCO (user's statement)
- 1× sighting with sighting-anecdote extension
- 1× event (user-reported)
- 1× impact (superseding previous assessment)
- Updated incident (with new evidence references)

Let's begin! 🚀

## Act 1: Setup - Scene 1: Import STIX Libraries

Same foundation as Step_2 - we're extending the existing incident.

In [2]:
import sys
!{sys.executable} -m pip install stixorm

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX libraries loaded")
print("✅ Act 1, Scene 1 complete")

✅ STIX libraries loaded
✅ Act 1, Scene 1 complete



[notice] A new release of pip is available: 24.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Act 1: Scene 2: Configure Python Path

In [3]:
import sys
sys.path.append('../')
import os

cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print("✅ Act 1, Scene 2 complete")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Act 1, Scene 2 complete


## Act 1: Scene 3: Import Utilities

We need the same utilities as Step_2, plus:
- `invoke_make_anecdote_block()`: Create anecdote SCO (user testimony)
- Functions to load and update the existing incident

In [4]:
import json
import os

# Incident management utilities
from Utilities.local_make_general import (
    invoke_save_incident_context_block,
    invoke_get_from_company_block,
    invoke_move_unattached_to_other_block,
    invoke_chain_sequence_block
)

# SDO creation utilities
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block,
    invoke_make_event_block,
    invoke_make_impact_block,
    invoke_make_task_block,
    invoke_make_sequence_block
)

# SCO creation utilities
from Utilities.local_make_sco import invoke_make_anecdote_block

# SRO creation utilities
from Utilities.local_make_sro import invoke_sighting_block, invoke_sro_block

# Paths
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"

# User report data
user_report = {
    "reporter": "CEO",
    "report_time": "2024-01-15T08:15:00Z",
    "statement": "I received a suspicious email claiming to be from IT support. It asked me to verify my account by clicking a link. I didn't click it - the sender address looked fake (attacker@evil.com).",
    "context": "User recognized phishing attempt and did not interact with malicious link"
}

print("✅ Utilities loaded")
print(f"✅ User report configured:")
print(f"   Reporter: {user_report['reporter']}")
print(f"   Time: {user_report['report_time']}")
print("✅ Act 1 complete - Ready to create anecdote!")

✅ Utilities loaded
✅ User report configured:
   Reporter: CEO
   Time: 2024-01-15T08:15:00Z
✅ Act 1 complete - Ready to create anecdote!


## Act 2: The User Report - Scene 1: Load Existing Context

Before creating the anecdote, we need to:
1. **Load the CEO's identity** (from Step_1 company context)
2. **Load your identity** (the investigator, from Step_0)

We'll reference these when creating the anecdote - showing WHO reported it and WHO is investigating.

In [5]:
# Load CEO identity from company context (created in Step_1)
reporter_name = user_report["reporter"]  # "CEO"
context_type = {"context_type": "users"}
reporter_query = {
    "type": "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}

try:
    reporter_identity = invoke_get_from_company_block(reporter_query, context_type, None, None)
    print(f"✅ CEO identity loaded: {reporter_identity.get('name', reporter_name)}")
except Exception as e:
    print(f"⚠️ Could not load CEO identity: {e}")
    reporter_identity = {"name": reporter_name, "id": "identity--placeholder"}

print("")
print("✅ Context loaded - ready to create anecdote!")
print("✅ Act 2, Scene 1 complete")

company query->{'type': 'identity', 'property': {'path': ['name'], 'source_value': 'CEO', 'comparator': 'EQ'}}
✅ CEO identity loaded: Naive Smith

✅ Context loaded - ready to create anecdote!
✅ Act 2, Scene 1 complete


### Scene 2: Create Anecdote SCO

**Pattern 3.9: Anecdote Provenance**

An **anecdote** is a new SCO type (from OS Threat extensions) that records human testimony:

```json
{
  "type": "anecdote",
  "statement": "User's description of what they saw/experienced",
  "made_by_ref": "identity--{ceo}",  // WHO said it
  "context_refs": ["observed-data--{original-email}"],  // WHAT they're talking about
  "timestamp": "2024-01-15T08:15:00Z"  // WHEN they said it
}
```

This creates **provenance** for human testimony - we know who said what, when, and about what.

Why this matters: In court or compliance audits, you can prove "This person reported this incident at this time."

In [6]:
# Create anecdote with user's statement
anecdote = invoke_make_anecdote_block(
    "SCO/Anecdote/anecdote_on_impact.json",
    "step3/impact_anecdote",
    anecdote_reporter=reporter_identity
)

# Save to incident context
anecdote_obj_path = results_base + "step3/impact_anecdote__anecdote.json"
anecdote_context_path = results_base + "step3/context/anecdote_context.json"
invoke_save_incident_context_block(anecdote_obj_path, anecdote_context_path)

print(f"✅ Anecdote created (Pattern 3.9)")
print(f"   Statement: \"{user_report['statement'][:60]}...\"")
print(f"   Made by: {reporter_identity.get('name', 'CEO')}")
print(f"   Time: {user_report['report_time']}")
print(f"✅ Type: {anecdote['type']}")
print(f"✅ ID: {anecdote['id'][:45]}...")
print("")
print("📝 User testimony recorded!")

{
    "type": "anecdote",
    "spec_version": "2.1",
    "id": "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f",
    "value": "I clicked on the link, and my laptop screen went weird",
    "report_date": "2025-11-19T06:07:56.095214Z",
    "provided_by_ref": "identity--11d42873-4689-4aea-803d-d9965cb254b8"
}
input data->{'stix_object': {'type': 'anecdote', 'spec_version': '2.1', 'id': 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f', 'value': 'I clicked on the link, and my laptop screen went weird', 'report_date': '2025-11-19T06:07:56.095214Z', 'provided_by_ref': 'identity--11d42873-4689-4aea-803d-d9965cb254b8'}}
from ports 
stix_object->{'type': 'anecdote', 'spec_version': '2.1', 'id': 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f', 'value': 'I clicked on the link, and my laptop screen went weird', 'report_date': '2025-11-19T06:07:56.095214Z', 'provided_by_ref': 'identity--11d42873-4689-4aea-803d-d9965cb254b8'}
context type->
✅ Anecdote created (Pattern 3.9)
   Statement: "I received 

### Scene 3: Wrap in Observed-Data

Just like in Step_2, we need to wrap the anecdote in **observed-data** to provide context: "This anecdote was observed at this time."

In [7]:
# Wrap anecdote in observed-data
observed_data = invoke_make_observed_data_block(
    "SDO/ObservedData/observation-alert.json",
    "step3/observation_anecdote",
    observation=[anecdote]
)

# Save to incident context
obs_obj_path = results_base + "step3/observation_anecdote"
obs_context_path = results_base + "step3/context/observation_anecdote_context.json"
invoke_save_incident_context_block(obs_obj_path, obs_context_path)

print(f"✅ Observed-data created")
print(f"   Contains: {len([anecdote])} object (anecdote)")
print(f"   Type: {observed_data['type']}")
print("")
print("👁️ Evidence package complete!")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--4d7d1d82-eb9d-4063-8725-36d91c000d4b",
    "created": "2025-11-19T06:07:56.349Z",
    "modified": "2025-11-19T06:07:56.349Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f"
    ]
}
input data->{'stix_object': {'type': 'observed-data', 'spec_version': '2.1', 'id': 'observed-data--4d7d1d82-eb9d-4063-8725-36d91c000d4b', 'created': '2025-11-19T06:07:56.349Z', 'modified': '2025-11-19T06:07:56.349Z', 'first_observed': '2020-10-19T01:01:01.000Z', 'last_observed': '2020-10-19T01:01:01.000Z', 'number_observed': 1, 'object_refs': ['anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f']}}
from ports 
stix_object->{'type': 'observed-data', 'spec_version': '2.1', 'id': 'observed-data--4d7d1d82-eb9d-4063-8725-36d91c000d4b', 'created': '2025-11-19T06:07:56.349Z', 'modified': '20

### Scene 4: Create Sighting-Anecdote

**Pattern 3.3 (Sighting Extensions)** - now with a different extension!

In Step_2, we used `sighting-alert` (automated SIEM detection).  
In Step_3, we use `sighting-anecdote` (human report).

```
Sighting (human verification):
├── sighting_of_ref → reporter_identity (WHO reported)
├── observed_data_refs → [observed-data with anecdote] (WHAT was reported)
└── extensions → sighting-anecdote (HOW it was reported - user portal)
```

Same pattern, different extension - showing the **source** of evidence matters.

In [8]:
# Create sighting with anecdote extension
sighting = invoke_sighting_block(
    "SRO/Sighting/sighting_anecdote.json",
    "step3/sighting_anecdote",
    observed=[observed_data],
    sighted=reporter_identity,
    where=[]  # Optional: could reference incident response portal system
)

# Save to incident context
sighting_obj_path = results_base + "step3/sighting_anecdote"
sighting_context_path = results_base + "step3/context/sighting_anecdote_context.json"
invoke_save_incident_context_block(sighting_obj_path, sighting_context_path, {"context_type": "unattached"})

print(f"✅ Sighting created (Pattern 3.3 - anecdote variant)")
print(f"   Sighting of: {reporter_identity.get('name', 'CEO')}")
print(f"   Detection method: User report (manual)")
print(f"   Extension: sighting-anecdote")
print("")
print("🎯 Human verification complete!")
print("✅ Act 2 complete - Moving to Impact Assessment")

<class 'dict'>
['sighting_form', 'observed_data_refs', 'sighting_of_ref']
{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--0d888e07-2475-4b2e-8a5a-7d7730bc7e61",
    "created": "2025-11-19T06:07:56.736Z",
    "modified": "2025-11-19T06:07:56.736Z",
    "count": 1,
    "sighting_of_ref": "identity--11d42873-4689-4aea-803d-d9965cb254b8",
    "observed_data_refs": [
        "observed-data--4d7d1d82-eb9d-4063-8725-36d91c000d4b"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-anecdote": {
            "person_name": "user-report",
            "person_context": "employee",
            "report_submission": "interview"
        }
    }
}
input data->{'stix_object': {'type': 'sighting', 'spec_version': '2.1', 'id': 'sighting--0d888e07-2475-4b2e-8a5a-7d7730bc7e61', 'created': '2025-11-19T06:07:56.736Z', 'modified': '2025-11-19T06:07:56.736Z', '

## Act 3: Impact Update - Evidence Changes the Story

Now we have TWO pieces of evidence pointing to the same incident:
1. **SIEM alert** (3:47 AM): "Malicious email detected"
2. **User report** (8:15 AM): "I received it but didn't click"

The second piece of evidence **changes the impact assessment**.

### Pattern 3.10: Impact Supersession Chain

**Initial assessment** (Step_2, based on SIEM alert):
- Severity: HIGH
- Assumption: "User might have clicked the link"
- Risk: Account compromise possible

**Updated assessment** (Step_3, based on user report):
- Severity: MEDIUM
- Fact: "User did NOT click the link"
- Risk: No compromise occurred

The new impact **supersedes** the old one. Pattern 3.10 creates a chain:

```
impact--{updated}
└── supersedes_refs → [impact--{initial}]
```

This preserves the **evolution of understanding**: we thought it was high risk, then learned it was medium risk.

In [9]:
# Create updated impact assessment (Pattern 3.10)
updated_impact = invoke_make_impact_block(
    "SDO/Impact/anecdote_impact.json",
    "step3/updated_impact"
    # This would include supersedes_refs pointing to the initial impact from Step_2
)

# Save to incident context
impact_obj_path = results_base + "step3/updated_impact"
impact_context_path = results_base + "step3/context/updated_impact_context.json"
invoke_save_incident_context_block(impact_obj_path, impact_context_path)

print(f"✅ Updated impact assessment created (Pattern 3.10)")
print(f"   Previous: HIGH severity (potential compromise)")
print(f"   Updated: MEDIUM severity (no click, no compromise)")
print(f"   Supersedes: Initial assessment from Step_2")
print("")

# Create event for user report
event = invoke_make_event_block(
    "SDO/Event/event_alert.json",
    "step3/event_user_report"
)

# Save to incident context
event_obj_path = results_base + "step3/event_user_report"
event_context_path = results_base + "step3/context/event_context.json"
invoke_save_incident_context_block(event_obj_path, event_context_path)

print(f"✅ Event created")
print(f"   Event type: user-reported")
print(f"   Timestamp: {user_report['report_time']}")
print("")
print("📊 Impact assessment updated!")
print("✅ Act 3 complete")

{
    "type": "impact",
    "spec_version": "2.1",
    "id": "impact--1858cf16-91bd-48aa-bad5-df9aa8d8ad25",
    "created": "2025-11-19T06:07:57.650Z",
    "modified": "2025-11-19T06:07:57.650Z",
    "impact_category": "availability",
    "criticality": 99,
    "description": "Laptop is stuffed",
    "end_time": "2025-11-19T06:07:57.650Z",
    "recoverability": "regular",
    "start_time": "2025-11-19T06:07:57.650Z",
    "extensions": {
        "availability": {
            "availability_impact": 99
        },
        "extension-definition--7cc33dd6-f6a1-489b-98ea-522d351d71b9": {
            "extension_type": "new-sdo"
        }
    }
}
input data->{'stix_object': {'type': 'impact', 'spec_version': '2.1', 'id': 'impact--1858cf16-91bd-48aa-bad5-df9aa8d8ad25', 'created': '2025-11-19T06:07:57.65Z', 'modified': '2025-11-19T06:07:57.65Z', 'impact_category': 'availability', 'criticality': 99, 'description': 'Laptop is stuffed', 'end_time': '2025-11-19T06:07:57.65Z', 'recoverability': 'regul

## G. Retrieve Laptop Asset from Company Context

Before creating the impact assessment, we need to retrieve the laptop asset that was affected from the company context (created in Step_1).

In [ ]:
print("💻 Retrieving laptop asset from company context...")

# Query for HP Laptop 1 from company assets
laptop_name = "HP Laptop 1"

# Setup context type to search in assets.json
context_type = {
    "context_type": "assets"
}

# Create query for laptop identity
laptop_ident_query = {
    "type": "identity",
    "property": {
        "path": ["name"],
        "source_value": laptop_name,
        "comparator": "EQ"
    }
}

# Retrieve laptop identity from company context
try:
    laptop_identity = invoke_get_from_company_block(laptop_ident_query, context_type, source_value=None, source_id=None)
    print(f"✅ Laptop asset found: {laptop_identity.get('name', laptop_name)}")
    print(f"   ID: {laptop_identity['id']}")
except Exception as e:
    print(f"⚠️ Could not load laptop asset: {e}")
    laptop_identity = None

print("")

## H. Create Impact Assessment

Create an **Impact object** to quantify the incident effects using template-driven parameters.

In [10]:
print("💥 Creating template-driven impact assessment...")

# Template-driven impact creation using quantified assessment
impact_path = "SDO/Impact/anecdote_impact.json"  # Data template
results_path = "step3/impact_assessment"         # Results path

# Configure impact quantification using template-driven parameters
numbers = {"computers-mobile": 1}  # Asset count impact
impacted_refs = [laptop_identity["id"]] if laptop_identity else []  # Specific affected assets

# Create impact using template-driven parameter generation
# Parameters are auto-generated from Impact template property types
impact_obj = invoke_make_impact_block(impact_path, results_path, 
                                    impacted_entity_counts=numbers, 
                                    impacted_refs=impacted_refs, 
                                    superseded_by_ref=None)

print(f"✅ Impact assessment created: {impact_obj['id']}")
print(f"   - Type: {impact_obj['type']}")
print(f"   - Impacted entity counts: {impact_obj.get('impacted_entity_counts', {})}")
print(f"   - Impacted refs: {impact_obj.get('impacted_refs', [])}")

# Save impact to incident context (automatic routing by STIX type - impact → impact_refs.json)
impact_results_obj_path = results_base + results_path
impact_results_context_path = results_base + "step3/context/impact_assessment_context.json"

result = invoke_save_incident_context_block(impact_results_obj_path, impact_results_context_path)
print(f"✅ Impact saved to context: {result}")

# Monitor context memory changes after impact creation (expect impact_refs.json)
print("\n📁 Context memory after impact creation:")
post_impact_files = monitor_context_memory()

# Compare with previous state - this should show impact_refs.json creation
new_files = set(post_impact_files) - set(post_sighting_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
    if 'impact_refs.json' in new_files:
        print("   ✅ EXPECTED: impact_refs.json created (category-based storage)")
else:
    print("   ℹ️ No new files created during impact creation")

💥 Creating template-driven impact assessment...


NameError: name 'laptop_identity' is not defined

## H. Template-Driven Task Management with Context Memory Monitoring

Create **investigation tasks** for the next steps using template-driven workflow management.

This operation creates the `task_refs.json` file in context memory with **automatic routing by STIX type**.

In [11]:
print("📋 Creating template-driven investigation task...")

# Template-driven task creation for investigation workflow
task_data_path = "SDO/Task/task_anecdote.json"  # Data template
results_path = "step3/task_anecdote"             # Results path

# Create task using template-driven parameter generation
# Parameters are auto-generated from Task template property types
task_obj = invoke_make_task_block(task_data_path, results_path, changed_objects=None)

print(f"✅ Investigation task created: {task_obj['id']}")
print(f"   - Type: {task_obj['type']}")
print(f"   - Name: {task_obj.get('name', 'N/A')}")
print(f"   - Description: {task_obj.get('description', 'N/A')}")

# Save task to incident context (automatic routing by STIX type - task → task_refs.json)
task_results_obj_path = results_base + results_path
task_results_context_path = results_base + "step3/context/task_anecdote_context.json"

result = invoke_save_incident_context_block(task_results_obj_path, task_results_context_path)
print(f"✅ Task saved to context: {result}")

# Monitor context memory changes after task creation (expect task_refs.json)
print("\n📁 Context memory after task creation:")
post_task_files = monitor_context_memory()

# Compare with previous state - this should show task_refs.json creation
new_files = set(post_task_files) - set(post_impact_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
    if 'task_refs.json' in new_files:
        print("   ✅ EXPECTED: task_refs.json created (category-based storage)")
else:
    print("   ℹ️ No new files created during task creation")

📋 Creating template-driven investigation task...
{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--f09dbeda-b888-4908-b04b-33be767b216e",
    "created": "2025-11-19T06:08:03.846Z",
    "modified": "2025-11-19T06:08:03.846Z",
    "description": "Find out who else got the email",
    "end_time": "2025-11-19T06:08:03.846Z",
    "name": "Query Exchange Server",
    "start_time": "2025-11-19T06:08:03.846Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}
✅ Investigation task created: task--f09dbeda-b888-4908-b04b-33be767b216e
   - Type: task
   - Name: Query Exchange Server
   - Description: Find out who else got the email
input data->{'stix_object': {'type': 'task', 'spec_version': '2.1', 'id': 'task--f09dbeda-b888-4908-b04b-33be767b216e', 'created': '2025-11-19T06:08:03.846Z', 'modified': '2025-11-19T06:08:03.846Z', 'description': 'Find out who else got the email', 'end_ti

NameError: name 'monitor_context_memory' is not defined

## I. Template-Driven Sequence Management with Context Memory Monitoring

Create a **Sequence object** to manage the investigation workflow using template-driven sequencing.

This operation creates the `sequence_refs.json` file in context memory with **automatic routing by STIX type**.

In [ ]:
print("🔗 Creating investigation sequence for user report handling...")

# Pattern 3.7: Sequence Workflow Orchestration
# This sequence chains to the existing workflow from Step_2

# Step 1: Create the sequence
sequence_data_path = "SDO/Sequence/sequence_anecdote.json"
results_path = "step3/sequence_task_anecdote"

seq_handle_report = invoke_make_sequence_block(
    sequence_data_path,
    results_path,
    step_type="single_step",
    sequence_type="task",
    sequenced_object=task_obj["id"],
    on_completion=None,
    on_success=None,
    on_failure=None,
    next_steps=None
)

print(f"✅ Sequence created: {seq_handle_report['id']}")
print(f"   - Step type: {seq_handle_report.get('step_type', 'N/A')}")
print(f"   - Sequence type: {seq_handle_report.get('sequence_type', 'N/A')}")

# Step 2: Chain to previous sequences from Step_2
# This automatically links this sequence to the end of Step_2's workflow
chain_results_path = results_base + "step3/chain_sequence_result.json"
invoke_chain_sequence_block(results_base + results_path, chain_results_path)

print(f"✅ Sequence chained to Step_2 workflow")

# Step 3: Save to incident context
sequence_results_obj_path = results_base + results_path
sequence_results_context_path = results_base + "step3/context/sequence_anecdote_context.json"

result = invoke_save_incident_context_block(
    sequence_results_obj_path,
    sequence_results_context_path
)

print(f"✅ Sequence saved to incident: {result}")
print("")
print("? Complete workflow now spans both notebooks:")
print("   Step_2: [start] → seq_alert → seq_validate → seq_determine →")
print("   Step_3: → seq_handle_report (this sequence)")
print("   All task sequences properly chained!")

## I.1: Create Task-to-Event Relationships

**Pattern 3.5**: Link the task to the objects it creates and impacts using SRO relationship objects.

In [1]:
print("🔗 Creating task relationships...")

# Relationship 1: task creates the event (user report event)
rel_task_creates_event = invoke_sro_block(
    "SRO/Relationship/relationship_creates.json",
    "step3/rel_task_creates_event",
    source=task_obj["id"],
    target=event_obj["id"],
    relationship_type="creates"
)

invoke_save_incident_context_block(
    results_base + "step3/rel_task_creates_event__rel.json",
    results_base + "step3/context/rel_task_creates_event_context.json"
)

print(f"✅ Relationship created: task_handle_report → creates → event_report")

# Relationship 2: task impacts the updated impact assessment
rel_task_impacts = invoke_sro_block(
    "SRO/Relationship/relationship_impacts.json",
    "step3/rel_task_impacts",
    source=task_obj["id"],
    target=impact_obj["id"],
    relationship_type="impacts"
)

invoke_save_incident_context_block(
    results_base + "step3/rel_task_impacts__rel.json",
    results_base + "step3/context/rel_task_impacts_context.json"
)

print(f"✅ Relationship created: task_handle_report → impacts → impact_updated")
print("")
print("📊 Task relationships established:")
print("   task_handle_report:")
print("   ├─ creates → event_report")
print("   └─ impacts → impact_updated")

🔗 Creating task relationships...


NameError: name 'invoke_sro_block' is not defined

## Summary - How Incidents Grow

### ✅ What We Added to the Incident

**New Evidence (from user report)**:
- 1× anecdote SCO (user's statement)
- 1× observed-data (wrapping anecdote)
- 1× sighting with sighting-anecdote extension
- 1× event (user-reported at 8:15 AM)
- 1× impact (updated assessment, superseding initial)

**Total objects in incident now**: ~30+ (Step_2's ~25 + Step_3's ~5)

### 📊 The Incident Before and After

**Before Step_3** (just SIEM alert):
```
incident
├── sighting_refs → [sighting-alert]  // 1 sighting
├── event_refs → [event-alert]        // 1 event
├── impact_refs → [impact-initial]    // 1 impact (HIGH severity)
└── other_object_refs → [...]
```

**After Step_3** (SIEM + user report):
```
incident
├── sighting_refs → [sighting-alert, sighting-anecdote]  // 2 sightings
├── event_refs → [event-alert, event-user-report]        // 2 events
├── impact_refs → [impact-initial, impact-updated]       // 2 impacts (chain)
└── other_object_refs → [..., anecdote, observed-data-2]
```

The incident **accumulated evidence** and **evolved**.

### 🎓 Patterns Demonstrated

1. **Pattern 3.9 (Anecdote Provenance)**: Human testimony with full attribution
   - WHO said it: CEO identity
   - WHAT they said: The statement text
   - WHEN: Timestamp
   - ABOUT WHAT: Context references to original evidence

2. **Pattern 3.10 (Impact Supersession Chain)**: How assessments evolve
   - Initial impact: HIGH (based on detection alone)
   - Updated impact: MEDIUM (based on user confirmation of no-click)
   - Supersession: New impact references old one
   - Preservation: Both assessments kept for audit trail

3. **Sighting Extension Variants**: Different evidence sources
   - `sighting-alert`: Automated SIEM detection
   - `sighting-anecdote`: Manual user report
   - Same pattern, different metadata

### 🔍 The Complete Evidence Timeline

```
Timeline of Evidence Accumulation:

3:47 AM - SIEM Alert Fires
├── Automated detection (sighting-alert)
├── Creates event-alert
├── Initial impact: HIGH
└── Incident created

8:15 AM - User Reports (5 hours later)
├── CEO submits report (anecdote)
├── Creates sighting-anecdote  
├── Creates event-user-report
├── Updated impact: MEDIUM (supersedes HIGH)
└── Incident extended
```

### ? Why This Matters: Multi-Source Intelligence

**Question**: "How do we know this was a real threat?"  
**Answer**: "We have TWO independent sources:"
1. Automated SIEM detection (objective pattern matching)
2. User confirmation (subjective human verification)

**Question**: "What was the actual impact?"  
**Answer**: "Follow the supersession chain:"
1. Initial assessment: HIGH risk (might have been compromised)
2. Updated assessment: MEDIUM risk (confirmed no interaction)

**Question**: "When did we learn it was safe?"  
**Answer**: "Follow event timeline:"
1. 3:47 AM: Threat detected
2. 8:15 AM: User confirmed no-click (5 hours later)

The graph **preserves the investigation story** as it unfolded in real-time.

### 🎯 Key Insight: Incidents Are Alive

This demonstrates the core concept: **Incidents are not static documents.**

They start small (a single alert) and **grow** as:
- New evidence arrives (user reports, forensics, threat intel)
- Impact assessments evolve (severity increases or decreases)
- Timeline expands (more events discovered)
- Response actions are taken (tasks created and completed)

The STIX graph **grows organically** as the investigation progresses.

### 🚀 What's Next?

You could continue extending this incident with:
- **Sighting-Enrichment**: Threat intelligence sources confirm it's a known campaign
- **Additional anecdotes**: Other employees report receiving similar emails
- **Forensic evidence**: Email server logs analyzed
- **Response completion**: Tasks marked as done, sequences completed

Each addition follows the same pattern: create objects, link them to the incident via `_refs` lists.

### 🎉 All 4 Notebooks Complete!

**Step_0**: Created YOUR identity (the analyst)  
**Step_1**: Created COMPANY context (organization, employees, systems)  
**Step_2**: Created initial INCIDENT (phishing alert, automated detection)  
**Step_3**: EXTENDED incident (user report, impact update)

You now understand:
- Identity management (Pattern 3.2)
- Company context (multi-tenant storage)
- Incident creation (Pattern 3.1)
- Evidence accumulation (Patterns 3.3, 3.9)
- Impact evolution (Pattern 3.10)
- Timeline building (Pattern 3.4)

**You're ready to model real-world cybersecurity incidents in STIX!** 🚀

---

**Congratulations!** You've completed the Brett Blocks incident management tutorial.